## Setup

In [35]:
# Loading packages and their components
import pandas as pd
import numpy as np
import pickle
# Setting Pandas options
# pd.options.display.max_rows = 999 # For debugging, can be removed later
pd.options.mode.chained_assignment = None  # Disabling the pandas chained assignment warnings

In [36]:
def import_and_preproc():
    # Read in the data
    dengue_features_train = pd.read_csv('data/dengue_features_train.csv')
    dengue_features_test = pd.read_csv('data/dengue_features_test.csv')
    dengue_labels_train = pd.read_csv('data/dengue_labels_train.csv')

    raw_data = [dengue_features_train, dengue_features_test, dengue_labels_train]
    
    # Splitting the data into a San Juan and an Iquitos part
    iq = []
    sj = []
    for item in raw_data:
        sj.append( item[item.city=='sj'] )
        iq.append( item[item.city=='iq'] )

    # Transferring the date column to the label part of the data
    sj[2] = sj[2].join(sj[0]['week_start_date'])
    iq[2] = iq[2].join(iq[0]['week_start_date'])

    # Converting the date column to datetime format
    for i in range(len(sj)):
        sj[i]['week_start_date'] = pd.to_datetime(sj[i]['week_start_date'], format='%Y-%m-%d')  
        iq[i]['week_start_date'] = pd.to_datetime(iq[i]['week_start_date'], format='%Y-%m-%d')
        
    # Putting the date as index
    for i in range(len(sj)):
        sj[i] = sj[i].set_index('week_start_date', drop=False)
        iq[i] = iq[i].set_index('week_start_date', drop=False)
        
    return list([sj[0], sj[1], sj[2], iq[0], iq[1], iq[2]])

data_subsets = import_and_preproc()

In [37]:
data_subsets[0]

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
week_start_date,,,,,,,,,,,,,,,,,,,,,
1990-04-30,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1990-05-07,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
1990-05-14,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
1990-05-21,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
1990-05-28,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-03-25,sj,2008,13,2008-03-25,0.077850,-0.039900,0.310471,0.296243,27.19,296.958571,...,7.55,74.247143,27.19,13.644286,2.885714,25.042857,5.785714,30.0,21.1,1.8
2008-04-01,sj,2008,14,2008-04-01,-0.038000,-0.016833,0.119371,0.066386,3.82,298.081429,...,3.67,74.600000,3.82,14.662857,2.714286,26.242857,6.814286,30.6,22.2,0.5
2008-04-08,sj,2008,15,2008-04-08,-0.155200,-0.052750,0.137757,0.141214,16.96,297.460000,...,35.00,75.027143,16.96,14.184286,2.185714,25.000000,5.714286,29.4,21.7,30.7


## Features in the dataset
City and date indicators
* `city` – City abbreviations: `sj` for San Juan and `iq` for Iquitos
* `week_start_date` – Date given in yyyy-mm-dd format

NOAA's GHCN daily climate data weather station measurements
* `station_max_temp_c` – Maximum temperature
* `station_min_temp_c` – Minimum temperature
* `station_avg_temp_c` – Average temperature
* `station_precip_mm` – Total precipitation
* `station_diur_temp_rng_c` – Diurnal temperature range

PERSIANN satellite precipitation measurements (0.25x0.25 degree scale)
* `precipitation_amt_mm` – Total precipitation

NOAA's NCEP Climate Forecast System Reanalysis measurements (0.5x0.5 degree scale)
* `reanalysis_air_temp_k` – Mean air temperature
* `reanalysis_relative_humidity_percen` – Mean relative humidity
* `reanalysis_specific_humidity_g_per_kg` – Mean specific humidity
* `reanalysis_precip_amt_kg_per_mm` – Total precipitation
* `reanalysis_max_air_temp_k` – Maximum air temperature
* `reanalysis_min_air_temp_k` – Minimum air temperature
* `reanalysis_avg_temp_k` – Average air temperature
* `reanalysis_tdtr_k` – Diurnal temperature range

Satellite vegetation - Normalized difference vegetation index (NDVI) - NOAA's CDR Normalized Difference Vegetation Index (0.5x0.5 degree scale) measurements
* `ndvi_se` – Pixel southeast of city centroid
* `ndvi_sw` – Pixel southwest of city centroid
* `ndvi_ne` – Pixel northeast of city centroid
* `ndvi_nw` – Pixel northwest of city centroid

## Missing value imputation
Since the environmental values for each week are assumed to follow seasonal patterns, they can not be simply replaced with the mean over the entire study. Intstead, missing values in these variables can be replaced with the mean value of the week before and after, or the week before and after that has no missing values.

In [38]:
environmental_vars = [
    'ndvi_ne',
    'ndvi_nw',
    'ndvi_se', 
    'ndvi_sw',
    'precipitation_amt_mm',
    'reanalysis_air_temp_k',
    'reanalysis_avg_temp_k',
    'reanalysis_dew_point_temp_k',
    'reanalysis_max_air_temp_k',
    'reanalysis_min_air_temp_k',
    'reanalysis_precip_amt_kg_per_m2',
    'reanalysis_relative_humidity_percent',
    'reanalysis_sat_precip_amt_mm',
    'reanalysis_specific_humidity_g_per_kg',
    'reanalysis_tdtr_k',
    'station_avg_temp_c',
    'station_diur_temp_rng_c',
    'station_max_temp_c',
    'station_min_temp_c',
    'station_precip_mm'
                     ]

In [39]:
def replace_missing(df, colnames):
    # Store the time index because the code below is index based and needs numbers
    date = df.index
    df = df.reset_index(drop=True)
    for colname in colnames:
        try: # because there are columns that do not occur in all subsets of the dataset
            miss_idx = df[df[colname].isnull()].index.tolist()
            for idx in miss_idx:
                    # Search the nearest week before the week with the missing value
                    # that itself has no missing value
                    before = df.iloc[:idx,:][colname].dropna().tail(1)
                    # The same but for the weeks after the missing value
                    after = df.iloc[idx:,:][colname].dropna().head(1)
                    # Replace the missing value with the mean
                    df[colname][idx] = np.mean([before, after])
        except:
            continue
    # Re-attach the time index and drop the auxiliary index
    df = df.set_index(date, drop=True)
    return df

In [40]:
# Applying the Imputation
for i in range(len(data_subsets)):
    data_subsets[i] = replace_missing(data_subsets[i], environmental_vars)

Check if there are still variables with missing values in our dataset.

In [41]:
# Check if there are still variables with missing values in our dataset.
for subset in data_subsets:
    print(subset.isnull().sum())
    print('---'*10)

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

## Feature editing
The temperature features from the NCEP Climate Forecast System Reanalysis and those of the weather station are in different units. To have the temperature features in the same units as those of the NCEP Climate Forecast System, the Reanalysis variables will be converted to degrees Celsius. For uniformity, the diurnal temperature range is converted from Celsius to Kelvin, as differences in temperature are expressed in Kelvin. Furthermore, the feature `precipitation_amt_mm` is removed as its values are identical to those of `reanalysis_precip_amt_kg_per_mm`. 

In [42]:
data_subsets[0]

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
week_start_date,,,,,,,,,,,,,,,,,,,,,
1990-04-30,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1990-05-07,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
1990-05-14,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
1990-05-21,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
1990-05-28,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-03-25,sj,2008,13,2008-03-25,0.077850,-0.039900,0.310471,0.296243,27.19,296.958571,...,7.55,74.247143,27.19,13.644286,2.885714,25.042857,5.785714,30.0,21.1,1.8
2008-04-01,sj,2008,14,2008-04-01,-0.038000,-0.016833,0.119371,0.066386,3.82,298.081429,...,3.67,74.600000,3.82,14.662857,2.714286,26.242857,6.814286,30.6,22.2,0.5
2008-04-08,sj,2008,15,2008-04-08,-0.155200,-0.052750,0.137757,0.141214,16.96,297.460000,...,35.00,75.027143,16.96,14.184286,2.185714,25.000000,5.714286,29.4,21.7,30.7


In [43]:
# compare 'reanalysis_sat_precip_amt_mm' and 'precipitation_amt_mm'
for i in range(len(data_subsets)):
    if data_subsets[i].shape[1] > 5:
        print(data_subsets[i][['reanalysis_sat_precip_amt_mm', 'precipitation_amt_mm']].sample(5))

                 reanalysis_sat_precip_amt_mm  precipitation_amt_mm
week_start_date                                                    
2001-05-14                              77.36                 77.36
1999-10-29                              67.94                 67.94
1992-09-16                              41.32                 41.32
1992-11-18                              43.44                 43.44
1995-09-17                              40.29                 40.29
                 reanalysis_sat_precip_amt_mm  precipitation_amt_mm
week_start_date                                                    
2008-09-16                              48.44                 48.44
2011-01-22                              10.11                 10.11
2008-07-29                              13.73                 13.73
2011-07-09                              70.37                 70.37
2009-09-17                               9.52                  9.52
                 reanalysis_sat_precip_amt_mm  p

In [44]:
for i in range(len(data_subsets)):
    if data_subsets[i].shape[1] > 5:
        data_subsets[i] = (
            data_subsets[i]
            .assign(month = lambda df: df.index.month)
            .assign(reanalysis_air_temp_c = lambda df: df['reanalysis_air_temp_k']-273.15)
            .assign(reanalysis_avg_temp_c = lambda df: df['reanalysis_avg_temp_k']-273.15)
            .assign(reanalysis_dew_point_temp_c = lambda df: df['reanalysis_dew_point_temp_k']-273.15)
            .assign(reanalysis_max_air_temp_c = lambda df: df['reanalysis_max_air_temp_k']-273.15)
            .assign(reanalysis_min_air_temp_c = lambda df: df['reanalysis_min_air_temp_k']-273.15)
            .rename(columns={'station_diur_temp_rng_c': 'station_diur_temp_rng_k'})
            .drop(['reanalysis_air_temp_k','reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k',
                   'reanalysis_min_air_temp_k','precipitation_amt_mm'], axis=1)
        )

## Adding the population data

This additional data is available from the Dengue Forecasting [website](https://dengueforecasting.noaa.gov/), from which the data provided by DrivenData originates.

In [45]:
def load_pop(filename):
    ser = (
        pd.read_csv(filename)
        .assign(year = lambda df: df.Year) # to have a same-name column with the other dataframes
        .assign(Year = lambda df: pd.to_datetime(df.Year, format='%Y'))
        .set_index('Year', drop=True)
    )
    return ser
sj_pop = load_pop('data/San_Juan_Population_Data.csv')
iq_pop = load_pop('data/Iquitos_Population_Data.csv')

In [46]:
def merge_pop(df, pop):
    merged = pd.merge(df, pop, how='left', on='year')
    merged = merged.rename(columns={'Estimated_population': 'population'})
    merged = merged.set_index('week_start_date', drop=True)
    return merged

In [47]:
# data_subsets[0] = merge_pop(data_subsets[0], sj_pop)

In [48]:
data_subsets[0] = merge_pop(data_subsets[0], sj_pop)
data_subsets[1] = merge_pop(data_subsets[1], sj_pop)
data_subsets[2] = data_subsets[2].set_index('week_start_date', drop=True)
data_subsets[3] = merge_pop(data_subsets[3], iq_pop)
data_subsets[4] = merge_pop(data_subsets[4], iq_pop)
data_subsets[5] = data_subsets[5].set_index('week_start_date', drop=True)

In [49]:
# Splitting the data into their parts
sj_features_train, \
sj_features_test, \
sj_labels_train, \
iq_features_train, \
iq_features_test, \
iq_labels_train = data_subsets

In [50]:
pickle.dump(data_subsets, open('cleaned_data.pickle', 'wb'))